# Cell lines analisys
## Preprocessing notebooks that makes expression matrix analysis for further ##

In [19]:
import polars as pl
from pathlib import Path
from pycomfort.files import *
from pycomfort import files
import pyarrow
import pandas as pd
from functional import seq
from typing import *
import functools
from pycomfort.files import *

In [12]:
import genomepy
from genotations import *
from genotations import ensembl
from genotations.genomes import *
from genotations.quantification import *

In [13]:
import sys

base = Path("..")
local = (base / "dashboard").resolve()
if local.exists():
    sys.path.insert(0, Path("..").absolute().as_posix())
    sys.path.insert(0, local)
    print(sys.path)
else:
    base = Path(".")
%load_ext autoreload
%autoreload 2

[PosixPath('/home/antonkulaga/sources/expressions-dashboard/dashboard'), '/home/antonkulaga/sources/expressions-dashboard/notebooks/..', '/home/antonkulaga/.local/share/JetBrains/Toolbox/apps/IDEA-U/ch-0/223.8617.56.plugins/python/helpers-pro/jupyter_debug', '/home/antonkulaga/.local/share/JetBrains/Toolbox/apps/IDEA-U/ch-0/223.8617.56.plugins/python/helpers/pydev', PosixPath('/home/antonkulaga/sources/expressions-dashboard/dashboard'), '/home/antonkulaga/sources/expressions-dashboard/notebooks/..', '/home/antonkulaga/sources/expressions-dashboard/notebooks', '/home/antonkulaga/sources/expressions-dashboard', '/home/antonkulaga/micromamba/envs/dashboard/lib/python310.zip', '/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10', '/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/site-packages', '/home/antonkulaga/micromamba/envs/dashboard/lib/python3.10/site-packages']
The autoreload extension is already loaded. T

In [24]:
from dashboard.config import *
from dashboard.prepare import *
from dashboard.models import *
from dashboard.cell_models import *

In [15]:
apply_settings()

## Setting up Paths

In [16]:
locations = Locations(base)

In [17]:
tprint(locations.cell_lines)

cell_lines
	cell_lines_toc.tsv


In [22]:
samples = Path("/home/antonkulaga/samples/CELL_LINES")
tprint(samples, max_depth=1)

CELL_LINES
	SKOV3
	EA.hy926
	PANC-1
	MelJuSo
	HEK293T
	B16F1
	CCD-1070Sk
	THP-1
	A549
	B16F10
	RAW


In [28]:
Cells(locations.cell_lines).toc_samples

Cell line name,URL,Species,Phenotype,Tissue,Samples,Modified lines,Owners,Sources,Commens
str,str,str,str,str,str,str,str,str,str
"""INS-1""","""https://www.cellosaurus.org/CVCL_0352""","""Rat""","""Celule Beta de insulinom ""","""Insulinom/Pancreas""","""SAMN07421988, SAMN07421989, SAMN07421987, SAMN07421986""",null,"""Molecular Cell Biology Department""",null,null
"""PANC-1 ""","""https://www.cellosaurus.org/CVCL_0480""","""Rat""","""Beta cells""","""Pancreas""","""SAMN10460093, SAMN06642004""",null,"""Molecular Cell Biology Department""",null,null
"""MelJuSo ""","""https://www.cellosaurus.org/CVCL_1403""","""Human""","""Celule de melanom uman faza VGP""","""Melanoma""","""SAMN07168087, SAMN07168088, SAMN07168089, SAMN07168090, SAMN07168091, SAMN07168092""",null,"""Molecular Cell Biology Department""",null,null
"""HEK293T""","""https://www.cellosaurus.org/CVCL_0063""","""Human""","""Celule umane embrionare de rinichi""","""Human kidney""","""SAMN22420271, SAMN22420269, SAMN22420270""",null,"""Molecular Cell Biology Department, Enzymology Department""",null,null
"""CCD-1070Sk""","""https://www.cellosaurus.org/CVCL_2332""",null,null,null,"""SAMN21374231, SAMN21374232, SAMN21374233, SAMN21374234 """,null,"""Molecular Cell Biology Department""",null,null
"""RAW 264.7""","""https://www.cellosaurus.org/CVCL_0493""","""Mouse""",null,null,"""SAMN32489934, SAMN32489935""",null,"""Molecular Cell Biology Department""",null,"""FAILED, need to redo"""
"""SKOV3""","""https://www.cellosaurus.org/CVCL_0532""","""Human""",null,null,"""SAMN20710030 , SAMN20710031, SAMN20710032 """,null,"""Molecular Cell Biology Department""",null,null
"""B16F10""","""https://www.cellosaurus.org/CVCL_0159""","""Mouse""",null,"""Melanoma""","""SAMN22137224, SAMN22137225, SAMN22137226""",null,"""Ligand-Receptor Interactions Department""",null,null
"""B16F1""","""https://www.cellosaurus.org/CVCL_0158""","""Mouse""",null,null,"""SAMN07340361, SAMN07340362, SAMN07340363""",null,"""Ligand-Receptor Interactions Department""",null,null


## Loading data